Task

Subtask B: Conspiracy Category Classification : a system must discriminate which conspiracy theory a post supports. We consider four possible conspiracies:

In this competition, specific competition participants will tackle the SECOND of these two subtasks.

Covid Conspiracy: It is related to narratives concerning vaccine production, 5G, restrictions, and all kind of news intended to support the idea that the pandemic and actions taken by the government were useless instruments of control of the population.

Qanon Conspiracy: Qanon supported the idea that a group of Satanic Cannibalist sex abusers conspired against former U.S. President Donald Trump during his term of office. This theory extended far over the original scope embodying other beliefs that support (among the others) hate against women and the idea that a powerful elite (led by public figures like Pope Francis, Queen Elizabeth, and Hillary Clinton ) is trying to organize a New World Order.

Flat Earth: It is a theory that claims that Earth is flat. This theory deals also with discussions about UFO.

Pro-Russia Conspiracy: This theory supports the idea that in the Ukraine-Russia war, Ukraine and NATO supported by nazists soldiers hold the responsability of the war.

In [ ]:
%pip install emoji sklearn pandas numpy nltk transformers torch matplotlib seaborn

Documentazioni:
emoji: https://carpedm20.github.io/emoji/docs/

Il dataset presenta una lista di messaggi/post raccolti su vari social, una delle caratteristiche di queste forme di comunicazioni sono le emoticon (o emoji) e la forma del messaggio in se.
Questo significa che un testo che vuole catturare l'attenzione avrà un "titolo" d'impatto e spesso in maiuscolo, in quanto l'attenzione dedicata a questo tipo di contenuti è poca e di carattere transitorio.

Inoltre il dataset presenta una quantità di dati inadatta all'addestramento di un qualsiasi classficatore superabile tramite tecniche di data augmentation, questa caratteristica è intriseca nel problema in quanto i trend [^1] durano molto poco nel tempo e per un sistema di riconoscimento dei contenuti fuorvianti è necessario essere in grado di adattarsi in breve a queste variazioni.

Il dataset ha dimensione eterogenea, il che potrebbe portare a esempi per il transformer dove sono presenti lunghe stringhe di filler.






Di seguito verranno proposte alcune tecniche di preprocessing e di data augmentation


[^1]: Per trend si indica un argomento che cattura l'attenzione del pubblico e che ha orizzonte temporale molto breve, di solito nell'ordine dei giorni.

Possibili tentativi  

Data augmentation
- prendere solo i caratteri ripetuti o parole maiuscole e emoticon dal testo e verificare la label, questo può funzionare in quanto spesso la comunicazione passa per l'impatto della forma  
- utilizzare altri modelli di elaborazione del linguaggio, chiedendo di modificare il testo mantenendone il significato, nascondendone parti o riformulare in modo più o meno aggressivo la frase


Data preprocessing  
- Aggiungere una colonna che conta la percentuale di caratteri ripetuti o parole maiuscole ed una colonna con la presenza o meno di un set di emoji, per poi normalizzare il dataset trasformando in minuscolo tutti i caratteri  
- Raccolta di una percentuale della frase per raccogliere il "titolo" 

In [1]:
import numpy
import pandas as pd


#carica il dataset csv dal file presente in Dataset/subtaskA_train.cvs
dataset = pd.read_csv('Dataset/subtaskB_train.csv',  header=0, names=['id', 'text', 'label'])

dataset

,id,text,label
0,"Siamo davanti ad una prova, e non solo di quoz...",Qanon,1
1,La storia dei 2 bimbi di Bergamo - vaccini: qu...,Covid,0
2,L'avete capito che non toglieranno il green pa...,Covid,0
3,Quindi la farsa dello spazio e della terra a ...,Terra Piatta,3
4,"In breve tempo, per accedere a Internet, sarà ...",Qanon,1
...,...,...,...
805,"Incredibile!!!! EMA, Agenzia Europea del Farma...",Covid,0
806,Non ci saranno colloqui di pace con la Russia ...,Russia,2
807,"L'atmosfera è uno ""scudo protettivo"" che avvol...",Terra Piatta,3
808,OTTIMA NOTIZIA! Due ragioni per cui le élite n...,Covid,0
